# Relevant Imports

In [1]:
from main_functions import *

# turn off divide-by-zero warnings since these null/NAN values get filtered out
import warnings
warnings.filterwarnings("ignore")

# load experimental data
data = pd.read_excel('data/data.xlsx', sheet_name = '1st Trial', names = ['time (min)', 'm_xylene', 'NO', 'NO2', 'NOx', 'NOx/m_xy', 'beta', 'NO,NO2 crossing time (min)', 'Wall loss factor', 'Peak O3 Concentration', '50% of Final M0 Time', 'PeakDp', 'deltaHC', 'm_xy consume ratio', 'deltaHC / beta', 'deltaHC * beta', 'M0', 'yield'])


# Summary Statistics

In [2]:
summary = np.round(data.describe(), 3)
with pd.ExcelWriter('written_data/No Transformations/summary.xlsx') as file:
    summary.to_excel(file, sheet_name = 'Statistics')

# Computing Correlations

### No transformations

In [3]:
# Determine end of input columns and end of intermediate columns
inp, inter = grab_column_split()

# Grab and color correlations
correlations = data.corr()
inter_correlations = np.round(correlations[data.columns[inp:inter]][:inp], 3)
output_correlations = np.round(correlations[data.columns[inter:]][:-len(data.columns[inter:])], 3)

inter_correlations = corr_highlight(inter_correlations)
output_correlations = corr_highlight(output_correlations)

In [4]:
# Hypothesis Testing for each computed correlation
pvals = data.corr(method = lambda x,y: 1 if (pearsonr(x, y)[1] < 0.05) else 0)
inter_pvals = pvals[data.columns[inp:inter]][:inp]
output_pvals = pvals[data.columns[inter:]][:-len(data.columns[inter:])]

# Highlight if significant
inter_pvals = pval_highlight(inter_pvals)
output_pvals = pval_highlight(output_pvals)

In [5]:
# Compute R-squared
cods = data.corr(method = lambda x,y: R2(x, y))
inter_cods = np.round(cods[data.columns[inp:inter]][:inp], 3)
output_cods = np.round(cods[data.columns[inter:]][:-len(data.columns[inter:])], 3)

# Highlight similarly to correlations
inter_cods = corr_highlight(inter_cods)
output_cods = corr_highlight(output_cods)

In [6]:
# Write to files
with pd.ExcelWriter('written_data/No Transformations/correlations.xlsx') as file:
    inter_correlations.to_excel(file, sheet_name = 'Intermediate Correlations')
    output_correlations.to_excel(file, sheet_name = 'Output Correlations')
    
with pd.ExcelWriter('written_data/No Transformations/pvalues.xlsx') as file:
    inter_pvals.to_excel(file, sheet_name = 'Intermediate P-Values')
    output_pvals.to_excel(file, sheet_name = 'Output P-Values')
    
with pd.ExcelWriter('written_data/No Transformations/cods.xlsx') as file:
    inter_cods.to_excel(file, sheet_name = 'Intermediate R2')
    output_cods.to_excel(file, sheet_name = 'Output R2')

### Input Values

In [7]:
X = data[data.columns[:inp]]
straight_corr = corr_highlight(np.round(X.corr(), 3))
straight_pval = pval_highlight(X.corr(method = lambda x,y: 1 if (pearsonr(x, y)[1] < 0.05) else 0))
straight_r2 = corr_highlight(np.round(X.corr(method = lambda x,y: R2(x, y)), 3))

In [8]:
with pd.ExcelWriter('written_data/No Transformations/inp_w_inp.xlsx') as file:
    straight_corr.to_excel(file, sheet_name = 'Correlations')
    straight_pval.to_excel(file, sheet_name = 'P-Values')
    straight_r2.to_excel(file, sheet_name = 'R2')

### Intermediate Values

In [9]:
X = data[data.columns[inp:inter]]
straight_corr = corr_highlight(np.round(X.corr(), 3))
straight_pval = pval_highlight(X.corr(method = lambda x,y: 1 if (pearsonr(x, y)[1] < 0.05) else 0))
straight_r2 = corr_highlight(np.round(X.corr(method = lambda x,y: R2(x, y)), 3))

In [10]:
with pd.ExcelWriter('written_data/No Transformations/inter_w_inter.xlsx') as file:
    straight_corr.to_excel(file, sheet_name = 'Correlations')
    straight_pval.to_excel(file, sheet_name = 'P-Values')
    straight_r2.to_excel(file, sheet_name = 'R2')

### Perform above steps for transformations and combinations of variables

In [11]:
f = lambda x, y: x + ' * ' + y
prod_corrs = functional_corr(data, (inp, inter), np.multiply, f, 2)

f = lambda x, y: x + ' / ' + y
prop_corrs = functional_corr(data, (inp, inter), np.divide, f, 2, 0)

f = lambda x: f'log({x})'
log_corrs = functional_corr(data, (inp, inter), np.log, f, 1)

f = lambda x: f'{x}^2'
sqr_corrs = functional_corr(data, (inp, inter), np.square, f, 1)

f = lambda x: f'{x}^3'
cube_corrs = functional_corr(data, (inp, inter), lambda x: np.power(x, 3), f, 1)

f = lambda x: f'sqrt({x})'
sqrt_corrs = functional_corr(data, (inp, inter), np.sqrt, f, 1)

corrs = [(prod_corrs, 'Product'), (prop_corrs, 'Proportion'), 
(log_corrs, 'Log'), (sqr_corrs, 'Square'), (cube_corrs, 'Cube'), (sqrt_corrs, 'Square Root')]

for corr in corrs:
    for i in range(3):
        correlations = corr[0]['correlations'][i]
        pvals = corr[0]['significance'][i]
        cods = corr[0]['coeff of determination'][i]
        
        corr_hl = corr_highlight(correlations)
        pval_hl = pval_highlight(pvals)
        cod_hl = corr_highlight(cods)
        
        corr[0]['correlations'][i] = corr_hl
        corr[0]['significance'][i] = pval_hl
        corr[0]['coeff of determination'][i] = cod_hl

### Save results

In [12]:
# Write correlations to files
with pd.ExcelWriter('written_data/Input w Intermediate Correlations/input_w_inter_corr_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['correlations'][0]
        correlations.to_excel(file, sheet_name = corr[1])

with pd.ExcelWriter('written_data/All w Output Correlations/final_corr_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['correlations'][2]
        correlations.to_excel(file, sheet_name = corr[1])


In [13]:
# Write hypothesis tests to files

with pd.ExcelWriter('written_data/Input w Intermediate Correlations/input_w_inter_pvals_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['significance'][0]
        correlations.to_excel(file, sheet_name = corr[1])

with pd.ExcelWriter('written_data/All w Output Correlations/final_pvals_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['significance'][2]
        correlations.to_excel(file, sheet_name = corr[1])


In [14]:
# Write R-squared to files

with pd.ExcelWriter('written_data/Input w Intermediate Correlations/input_w_inter_R2_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['coeff of determination'][0]
        correlations.to_excel(file, sheet_name = corr[1])

with pd.ExcelWriter('written_data/All w Output Correlations/final_R2_.xlsx') as file:
    for corr in corrs:
        correlations = corr[0]['coeff of determination'][2]
        correlations.to_excel(file, sheet_name = corr[1])